In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [ ]:
path = ''
df_all_tweet = pd.read_csv(path + '/Dataset/PolitiX_Final_Tweet_Dataset.csv', dtype = 'str', lineterminator = '\n')

In [ ]:
df_all_tweet['created_at_date'] = pd.to_datetime(df_all_tweet['created_at'], format='%a %b %d %H:%M:%S %z %Y')


In [ ]:
def predict_user_timezone(tweet_times_utc):    
    offset_range = range(-12, 15)
    awake_start = 7
    awake_end = 22

    offset_scores = {}

    for offset in offset_range:
        shifted = tweet_times_utc + pd.to_timedelta(offset, unit='h')
        local_hours = shifted.dt.hour
        awake_count = np.sum((local_hours >= awake_start) & (local_hours <= awake_end))
        offset_scores[offset] = awake_count

    sorted_offsets = sorted(offset_scores.items(), key=lambda x: x[1], reverse=True)
    top_3_offsets = sorted_offsets[:3]
    result = [f"UTC{o[0]:+d}" for o in top_3_offsets]
    return result




In [ ]:
df_all_tweet['created_at_date']

In [ ]:

sample = df_all_tweet.loc[df_all_tweet.User_ID == '1424619313'].created_at_date
predicted_timezones = predict_user_timezone(sample)
print(f"Top 3 most likely timezones: {predicted_timezones}")


In [ ]:
df_all_tweet.sample(1)

In [ ]:
df_all_tweet.groupby('User_ID').count()

In [ ]:
df_all_tweet.User_ID.unique()

In [ ]:
user_time = {}

In [ ]:
user_time = {}
for user_id, group in df_all_tweet.groupby('User_ID'):
    predicted_timezones = predict_user_timezone(group['created_at_date'])
    user_time[user_id] = predicted_timezones

In [ ]:
# Create DataFrame directly from the dictionary
df_user_time = pd.DataFrame.from_dict(user_time, orient='index', 
                                     columns=['timezone_1', 'timezone_2', 'timezone_3'])

# Reset index to make User_ID a column
df_user_time = df_user_time.reset_index().rename(columns={'index': 'User_ID'})

In [ ]:
df_user_time

In [ ]:
df_user_time.loc[df_user_time.timezone_1  == 'UTC+8'].sample(5)

In [ ]:
import pandas as pd
import glob
import os

# Read each foreign countries user list
folder_path = ''
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
df_list = []
for file in csv_files:
    df = pd.read_csv(file)
    df['source_file'] = os.path.basename(file) 
    df_list.append(df)
combined_df = pd.concat(df_list, ignore_index=True)
print(combined_df.head())

In [ ]:
combined_df['country'] = combined_df['source_file'].str.replace('_ID.csv', '', regex=False)

In [ ]:
combined_df.loc[combined_df.country == 'Russian']

In [ ]:
combined_df.groupby('country').count()

In [ ]:
df_user_time.User_ID = df_user_time.User_ID.astype('int')

In [ ]:
df_user_time.loc[df_user_time.User_ID.isin(combined_df.loc[combined_df.country == 'Persian'].User_ID)]

In [ ]:
len(df_user_time.loc[df_user_time.User_ID.isin(combined_df.loc[combined_df.country == 'Chinese'].User_ID)])

In [ ]:
df_user_time.loc[df_user_time.timezone_1 == 'UTC-5']

In [ ]:
df_user_time.loc[df_user_time.timezone_2 == 'UTC-5']

In [ ]:
import pandas as pd
target_timezones = [f'UTC-{i}' for i in range(2, 13)]
mask = (
    df_user_time['timezone_1'].isin(target_timezones) &
    df_user_time['timezone_2'].isin(target_timezones) &
    df_user_time['timezone_3'].isin(target_timezones)
)

num_users = df_user_time[mask]['User_ID'].nunique()

print(f"Number of accounts with timezone between UTC-2 and UTC-12: {num_users}")


In [ ]:
import pandas as pd
target_timezones = [f'UTC+{i}' for i in range(0, 12)]  # ['UTC+2', ..., 'UTC+11']

mask_all = (
    df_user_time['timezone_1'].isin(target_timezones) &
    df_user_time['timezone_2'].isin(target_timezones) &
    df_user_time['timezone_3'].isin(target_timezones)
)
num_users_all = df_user_time[mask_all]['User_ID'].nunique()

print(f"Number of accounts where ALL timezones are between UTC+2 and UTC+11: {num_users_all}")


In [ ]:
num_users_all

In [ ]:
import pandas as pd

target_timezones = [f'UTC+{i}' for i in range(2, 5)]
mask_all = (
    df_user_time['timezone_1'].isin(target_timezones) &
    df_user_time['timezone_2'].isin(target_timezones) &
    df_user_time['timezone_3'].isin(target_timezones)
)

num_users_all = df_user_time[mask_all]['User_ID'].nunique()

print(f"Number of accounts where ALL timezones are between UTC+2 and UTC+11: {num_users_all}")


In [ ]:
df_user_time[mask_all]['User_ID'].sample(10)

In [ ]:
combined_df

In [ ]:
df_foeign_timezone = df_user_time[mask_all]

In [ ]:
df_foeign_timezone[~df_foeign_timezone.User_ID.isin(combined_df.User_ID)]

In [ ]:
df_foeign_timezone.loc[df_foeign_timezone.User_ID.isin(russina_lbp)]

In [ ]:
df_user_time.loc[df_user_time.User_ID.isin(russina_lbp)]

In [ ]:
import pandas as pd
target_timezones = [f'UTC+{i}' for i in range(2, 5)]
mask_all = (
    df_user_time['timezone_1'].isin(target_timezones) &
    df_user_time['timezone_2'].isin(target_timezones) &
    df_user_time['timezone_3'].isin(target_timezones)
)
num_users_all = df_user_time[mask_all]['User_ID'].nunique()
print(f"Number of accounts where ALL timezones are between UTC+2 and UTC+11: {num_users_all}")

In [ ]:
df_foeign_timezone[mask_all].sample(20)

# Foreign Suspected

In [ ]:
df_lbp_foreign = pd.read_csv('Suspected_LBP_Foreign.csv')

In [ ]:
df_lbp_foreign['User_ID'] = df_lbp_foreign['Inferred_User_ID']

In [ ]:
combined_df.country.unique()

In [ ]:
df_foreign_sus = pd.read_csv('Foreign_Supspected.csv')

In [ ]:
df_foreign_sus.loc[df_foreign_sus.Country == 'portuguese']

In [ ]:
df_foreign_sus.Country.unique()

In [ ]:
country_timezones = {
    'Chinese': 'UTC+08:00',      # China Standard Time (CST)
    'Israeli': 'UTC+02:00',      # Israel Standard Time (IST)
    'Portuguese': 'UTC+00:00',   # Portugal (mainland), Azores is UTC−01:00
    'Ukranian': 'UTC+02:00',     # Ukraine Standard Time
    'Persian': 'UTC+03:30',      # Iran Standard Time (IRST)
    'Russian': 'UTC+03:00',      # Moscow Standard Time (varies across Russia)
    'Spanish': 'UTC+01:00',      # Central European Time (Spain mainland)
    'India': 'UTC+05:30'         # Indian Standard Time (IST)
}


In [ ]:
df_foeign_timezone

In [ ]:
# import pandas as pd

# # Sample UTC timezone dictionary
# country_timezones = {
#     'Chinese': 'UTC+08:00',
#     'Israeli': 'UTC+02:00',
#     'Portuguese': 'UTC+00:00',
#     'Ukranian': 'UTC+02:00',
#     'Persian': 'UTC+03:30',
#     'Russian': 'UTC+03:00',
#     'Spanish': 'UTC+01:00',
#     'India': 'UTC+05:30'
# }

# # Function to convert UTC string to float hours
# def utc_str_to_offset(utc_str):
#     sign = 1 if '+' in utc_str else -1
#     h, m = map(int, utc_str.replace('UTC+', '').replace('UTC-', '').split(':'))
#     return sign * (h + m / 60)

# # Convert country UTCs to numeric offsets
# country_offsets = {k: utc_str_to_offset(v) for k, v in country_timezones.items()}

# # Merge the two dataframes on User_ID
# df = df_lbp_foreign.merge(df_foeign_timezone, on='User_ID')

# # Function to check if any predicted timezone is within ±3 hours
# def is_within_3_hours(row):
#     country = row['Country']
#     base_offset = country_offsets.get(country)
#     if base_offset is None:
#         return False  # Skip unknown countries
#     for col in ['timezone_1', 'timezone_2', 'timezone_3']:
#         val = row[col]
#         if isinstance(val, str) and val.startswith('UTC'):
#             offset = utc_str_to_offset(val)
#             if abs(offset - base_offset) <= 3:
#                 return True
#     return False

# # Apply function to check if timezones are within ±3 hours
# df['within_3_hours'] = df.apply(is_within_3_hours, axis=1)

# # Count how many accounts satisfy the condition
# num_matching_accounts = df['within_3_hours'].sum()

# print(f"Number of loopy belief accounts with timezone within ±3 hours of country UTC: {num_matching_accounts}")


In [ ]:
#df.groupby('Country').count()

In [ ]:
df.groupby('Country').count()

In [ ]:
df_lbp_foreign.loc[df_lbp_foreign.Country == 'India']

In [ ]:
df_user_time.loc[df_user_time.User_ID.isin(df_lbp_foreign.loc[df_lbp_foreign.Country == 'India'].User_ID)]

In [ ]:
user_foreing_timezone = df_all_tweet.loc[df_all_tweet.User_ID.isin(df_foeign_timezone.User_ID.astype('str'))]

In [ ]:
user_foreing_timezone['is_retweet'] = user_foreing_timezone['text'].str.startswith('RT @')


In [ ]:
user_foreing_timezone_tweet = user_foreing_timezone.loc[user_foreing_timezone.is_retweet == False]

In [ ]:
user_foreing_timezone.Text_Lang.unique()

In [ ]:
for i in user_foreing_timezone_tweet.Text_Lang.unique():
     print(f"Language {i} has {len(user_foreing_timezone_tweet.loc[user_foreing_timezone_tweet.Text_Lang == i].User_ID.unique())} users")
    

In [ ]:
# Step 1: Count tweets per user-language pair
user_lang_counts = user_foreing_timezone_tweet.groupby(['User_ID', 'Text_Lang']).size().reset_index(name='Tweet_Count')

# Step 2: Keep only user-language pairs with at least 2 tweets
at_least_two = user_lang_counts[user_lang_counts['Tweet_Count'] >= 2]

# Step 3: Count unique users per language
lang_user_counts = (
    at_least_two.groupby('Text_Lang')['User_ID']
    .nunique()
    .reset_index(name='User_Count')
    .sort_values('User_Count')
)

# Step 4: Print sorted results
for _, row in lang_user_counts.iterrows():
    print(f"Language {row['Text_Lang']} has {row['User_Count']} users with at least 2 tweets")


In [ ]:
user_lang_counts = user_foreing_timezone_tweet.groupby(['User_ID', 'Text_Lang']).size().reset_index(name='Tweet_Count')
at_least_two = user_lang_counts[user_lang_counts['Tweet_Count'] >= 2]
language_users = {}
for lang, group in at_least_two.groupby('Text_Lang'):
    language_users[lang] = group['User_ID'].unique().tolist()

lang_user_counts = (
    pd.DataFrame({
        'Text_Lang': list(language_users.keys()),
        'User_Count': [len(users) for users in language_users.values()]
    })
    .sort_values('User_Count', ascending=False)
    .reset_index(drop=True)
)

print("Language distribution of users with at least 2 tweets:")
for idx, row in lang_user_counts.iterrows():
    lang = row['Text_Lang']
    count = row['User_Count']
    print(f"Language {lang}: {count} users")


In [ ]:
len(language_users)

In [ ]:
len(user_foreing_timezone_tweet.User_ID.unique())

In [ ]:
user_ids = []
languages = []
for lang in ['da', 'de', 'fr', 'ar', 'tr', 'ur', 'it']:
    for user_id in language_users[lang]:
        user_ids.append(user_id)
        languages.append(lang)

multilingual_users_df = pd.DataFrame({
    'User_ID': user_ids,
    'Language': languages
})

print(multilingual_users_df.head())

print(f"\nTotal records: {len(multilingual_users_df)}")
print(f"Users by language:")
print(multilingual_users_df['Language'].value_counts())

In [ ]:
df_f_manual = pd.read_csv('Foreign_Supspected.csv')

In [ ]:
import pandas as pd

# Define the target UTC+ range
target_timezones = [f'UTC+{i}' for i in range(1, 12)]

mask_all = (
    df_user_time['timezone_1'].isin(target_timezones) |
    df_user_time['timezone_2'].isin(target_timezones) |
    df_user_time['timezone_3'].isin(target_timezones)
)

num_users_all = df_user_time[mask_all]['User_ID'].nunique()

print(f"Number of accounts where ALL timezones are between UTC+2 and UTC+11: {num_users_all}")


In [ ]:
pd.merge(df_user_time[mask_all], df_f_manual)[['User_ID', 'Country']]

In [ ]:
df_lbp = pd.read_csv('Suspected_LBP_Foreign.csv')

In [ ]:
pd.concat([multilingual_users_df.rename(columns = {'Language': 'Country'}), 
           pd.merge(df_user_time[mask_all], df_f_manual)[['User_ID', 'Country']], 
          df_lbp.rename(columns = {'Inferred_User_ID': 'User_ID'})]).to_csv('Final_Foreign.csv', index = False)

In [ ]:
df_lbp.rename(columns = {'Inferred_User_ID': 'User_ID'})